# トピックモデル（LDA）

In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

## 訓練データ

In [2]:
# 9つの文書 (11単語)
docs = ["dog cute cat cute",
            "dog cat cute",
            "cute dog cute cat",
            "soccer team fan",
            "baseball team soccer team",
            "baseball fan soccer fan",
            "Japan China U.S.",
            "China large U.S. large",
            "China large Japan"]

### 前処理（Bag of Words化）

文書を各単語の集合にする

In [3]:
# 文書を単語のリストにする
_docs = []
for doc in docs:
    _docs.append(doc.split())

words = []
dic = {}
bw = []
for _doc in _docs:
    ws = []
    for word in _doc:
        if word in words:
            ws.append(words.index(word))
        else:
            words.append(word)
            id_ = words.index(word)
            ws.append(id_)
            dic[id_] = word
    bw.append(ws)

dic, bw

({0: 'dog',
  1: 'cute',
  2: 'cat',
  3: 'soccer',
  4: 'team',
  5: 'fan',
  6: 'baseball',
  7: 'Japan',
  8: 'China',
  9: 'U.S.',
  10: 'large'},
 [[0, 1, 2, 1],
  [0, 2, 1],
  [1, 0, 1, 2],
  [3, 4, 5],
  [6, 4, 3, 4],
  [6, 5, 3, 5],
  [7, 8, 9],
  [8, 10, 9, 10],
  [8, 10, 7]])

## Latent Dirichlet Allocation(LDA)のモデル

$\boldsymbol{\theta}_{i}$を文書$i$のトピック分布、$\boldsymbol{\phi}_{k}$をトピック$k$の単語分布、$z_{ij}$を文書$i$中の単語$j$に割り当てられるトピック、  
$w_{ij}$を文書$i$中の単語$j$、$D$を文書数、$K$をトピック数、$N$を単語数、$\alpha, \beta$をディリクレ分布のハイパーパラメータとすると、  
文書の生成過程は以下のとおりである。

1. $\boldsymbol{\theta}_{i} \sim \text{Dir}(\alpha) \qquad (i=1, 2, ..., D)$
2. $\boldsymbol{\phi}_{k} \sim \text{Dir}(\beta) \qquad (K=1, 2, ..., K)$
3. 各文書$i \ (i=1, 2, ..., D)$と単語$j \ (j=1, 2, ..., N_{i})$に対して、  
     a) $z_{ij} \sim \text{Multinomial}(\boldsymbol{\theta}_{i})$  
     b) $w_{ij} \sim \text{Multinomial}(\boldsymbol{\phi}_{z_{ij}})$
     
ただし、$\text{Dir}(\cdot)$はディリクレ分布、$\text{Multinomial}(\cdot)$は多項分布である。

## 推論

トピックモデルではある単語$w$が与えられたときのトピック$z$の事後確率$p(z | w)$を知る必要がある。

しかし、これは解析的に解くことができないので、今回は崩壊型ギブスサンプリングを用いて事後分布を推定する。

崩壊型ギブスサンプリングとは、確率モデルから一部の確率変数を周辺化除去することでサンプリング効率を高めたギブスサンプリング（ある確率変数をサンプリングする際、その他の確率変数を固定した確率分布からサンプリングする手法）である。

文書全体$W$が与えられたとき、同時分布は以下のようになる。

$p(Z, W, \boldsymbol{\theta}, \boldsymbol{\phi}; \alpha, \beta) = \prod_{k=1}^{K} p(\boldsymbol{\phi}_{i}; \beta) \prod_{i=1}^{D} p(\boldsymbol{\theta}_{i}; \alpha) \prod_{j=1}^{N_{i}} p(Z_{ij} | \boldsymbol{\theta}_{i}) p(W_{ij} | \boldsymbol{\phi}_{z_{ij}})$

計算の詳細は省くが、パラメータ$\boldsymbol{\theta}, \boldsymbol{\phi}$を周辺化除去することで、$Z_{ij}$の事後分布は以下に従う。

\begin{eqnarray}
p(Z_{ij}=k | Z_{-(ij)}, W; \alpha, \beta) &\propto& p(Z_{ij}=k, Z_{-(ij)}, W; \alpha, \beta) \\
&\propto& (N_{ik(\cdot)}^{-(ij)} + \alpha) \frac{N_{(\cdot)kj}^{-(ij)} + \beta}{\sum_{r=1}^{N_{i}} (N_{(\cdot)kr}^{-(ij)} + \beta)}
\end{eqnarray}

ただし、$Z_{-(ij)}$は$Z$から$Z_{ij}$を除いたものである。また$N_{ik(\cdot)}^{-(ij)}$は文書$i$中の単語$j$を除いた上での文書$i$中のトピック$k$の単語数、$N_{(\cdot)kj}^{-(ij)}$は文書$i$中の単語$j$を除いた上での全文書中でのトピック$k$の単語$j$の数を表す。

In [4]:
n_words = len(words)
n_docs = len(bw)
n_topics = 3

# Zの初期値は一様サンプリング
Z = [[np.random.randint(n_topics) for _ in bw[i]] for i in range(n_docs)]

# 文書i内で、トピックkに割り当てられた単語数
Nik = np.zeros((n_docs, n_topics), dtype=int)
# トピックkに割り当てられた単語jの数
Nkj = np.zeros((n_topics, n_words), dtype=int)
for i in range(n_docs):
    for w, z in zip(bw[i], Z[i]):
        Nik[i, z] += 1
        Nkj[z, w] += 1

In [5]:
alpha = 0.1
beta = 0.01

def conditional_distribution(i, j):
    # p(z=k | Z, W)
    left = Nik[i] + alpha
    right = (Nkj[:, j] + beta) / (Nkj.sum(axis=1) + beta*n_words)
    
    p_z = left * right
    return p_z / np.sum(p_z)

def sample_z(p):
    return np.random.choice(n_topics, p=p)

max_iter = 10
for _ in range(max_iter):
    for i in range(n_docs):
        for j, (w, z) in enumerate(zip(bw[i], Z[i])):
            # 該当の単語を抜く
            Nik[i, z] -= 1
            Nkj[z, w] -= 1

            p_z = conditional_distribution(i, w)
            z_new = sample_z(p_z)
            
            # 更新
            Z[i][j]= z_new
            Nik[i, z_new] += 1
            Nkj[z_new, w] += 1

## 結果

事後分布$p(Z | W)$からのサンプリング結果

In [6]:
Z

[[0, 0, 0, 0],
 [0, 0, 0],
 [0, 0, 0, 0],
 [2, 2, 2],
 [2, 2, 2, 2],
 [2, 2, 2, 2],
 [1, 1, 1],
 [1, 1, 1, 1],
 [1, 1, 1]]